<a href="https://colab.research.google.com/github/MAHESWARAREDDYAARIKATLA/Daily-renewable-generation-data-/blob/main/Daily_Renewable_energy_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from IPython import get_ipython
from IPython.display import display

# Install required libraries
!apt-get update
!apt-get install -y chromium-chromedriver
!pip install selenium requests pandas openpyxl

import os
import re
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Selenium WebDriver setup
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(options=chrome_options)

download_folder = "/content/excel_reports"
output_folder = "/content/cleaned_reports"
os.makedirs(download_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

# Base URL
base_url = "https://cea.nic.in/daily-renewable-generation-report/?lang=en"

def navigate_and_scrape_all_pages():
    print(f"Navigating to {base_url}...")
    driver.get(base_url)

    all_links = set()

    while True:
        # Wait for the page to load and fetch Excel file links
        try:
            links = WebDriverWait(driver, 20).until(
                EC.presence_of_all_elements_located((By.XPATH, "//a[contains(@href, '.xlsx')]"))
            )
            # Collect all unique Excel file links
            for link in links:
                file_url = link.get_attribute("href")
                if file_url:
                    all_links.add(file_url)
        except Exception as e:
            print(f"Error while fetching links: {e}")
            break

        # Try to find the "Next" button and navigate to the next page if available
        try:
            next_button = driver.find_element(By.XPATH, "//a[contains(text(), 'Next')]")
            next_button.click()
            WebDriverWait(driver, 5).until(
                EC.staleness_of(links[-1])  # Wait until the current page changes
            )
        except Exception:
            print("No more pages to navigate.")
            break

    print(f"Found {len(all_links)} unique download links.")
    return list(all_links)

def download_excel_files(file_urls):
    for file_url in file_urls:
        filename = os.path.join(download_folder, file_url.split("/")[-1])
        print(f"Downloading: {file_url}")
        try:
            response = requests.get(file_url)
            if response.status_code == 200:
                with open(filename, "wb") as file:
                    file.write(response.content)
                print(f"Downloaded: {filename}")
            else:
                print(f"Failed to download {file_url} (Status Code: {response.status_code})")
        except Exception as e:
            print(f"Error downloading {file_url}: {e}")

def clean_and_process_excel(file_path):
    try:
        # Load the Excel file
        data = pd.read_excel(file_path, sheet_name="Generation")
        cleaned_data = data.iloc[4:].copy()
        cleaned_data.columns = cleaned_data.iloc[0]
        cleaned_data = cleaned_data[1:]
        cleaned_data.dropna(how="all", axis=1, inplace=True)
        cleaned_data.dropna(how="all", axis=0, inplace=True)
        cleaned_data.columns = [str(col).strip().replace("-", "_").replace(r"\W+", "_").lower() for col in cleaned_data.columns]

        # Rename columns
        cleaned_data.columns = [
            "state_region",
            "wind_energy_mw",
            "solar_energy_mw",
            "other_res_mw",
            "total_mw",
            "cumulative_wind_mw",
            "cumulative_solar_mw",
            "cumulative_other_res_mw",
            "cumulative_total_mw",
        ]

        # Remove Hindi text and convert to numeric where possible
        hindi_pattern = re.compile(r"[\u0900-\u097F]")
        cleaned_data = cleaned_data.applymap(
            lambda x: re.sub(hindi_pattern, "", str(x)) if isinstance(x, str) else x
        )

        numeric_columns = [
            "wind_energy_mw",
            "solar_energy_mw",
            "other_res_mw",
            "total_mw",
            "cumulative_wind_mw",
            "cumulative_solar_mw",
            "cumulative_other_res_mw",
            "cumulative_total_mw",
        ]
        cleaned_data[numeric_columns] = cleaned_data[numeric_columns].apply(
            pd.to_numeric, errors="coerce"
        )
        cleaned_data.reset_index(drop=True, inplace=True)

        # Save cleaned data
        csv_file_name = os.path.join(output_folder, os.path.basename(file_path).replace(".xlsx", "_cleaned.csv"))
        cleaned_data.to_csv(csv_file_name, index=False)
        print(f"Processed and saved: {csv_file_name}")
    except Exception as e:
        print(f"Error processing {file_path}: {e}")

def process_all_files():
    excel_files = [f for f in os.listdir(download_folder) if f.endswith(".xlsx")]
    for file_name in excel_files:
        file_path = os.path.join(download_folder, file_name)
        clean_and_process_excel(file_path)

# Execute the entire workflow
file_urls = navigate_and_scrape_all_pages()
download_excel_files(file_urls)
process_all_files()

print("Data cleaning complete. Cleaned files are saved in the 'cleaned_reports' folder.")
driver.quit()


Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Fetched 259 kB in 1s (208 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading pa

<ipython-input-16-c9e483eb357d>:109: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cleaned_data = cleaned_data.applymap(
<ipython-input-16-c9e483eb357d>:109: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cleaned_data = cleaned_data.applymap(
<ipython-input-16-c9e483eb357d>:109: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cleaned_data = cleaned_data.applymap(
<ipython-input-16-c9e483eb357d>:109: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cleaned_data = cleaned_data.applymap(
<ipython-input-16-c9e483eb357d>:109: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cleaned_data = cleaned_data.applymap(
<ipython-input-16-c9e483eb357d>:109: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cleaned_data = cleaned_data.applymap(


Processed and saved: /content/cleaned_reports/25_Feb_2022_Daily_Report_cleaned.csv
Processed and saved: /content/cleaned_reports/8_Sept_2024_Daily_RE_Generation_Report_cleaned.csv
Processed and saved: /content/cleaned_reports/7_Oct_2024_Daily_RE_Generation_Report_cleaned.csv
Processed and saved: /content/cleaned_reports/23_Feb_2022_Daily_Report_cleaned.csv
Processed and saved: /content/cleaned_reports/28_May_2023_Daily_Report_cleaned.csv
Processed and saved: /content/cleaned_reports/23_Jan_2023_Daily_Report_cleaned.csv
Error processing /content/excel_reports/22_July.xlsx: Worksheet named 'Generation' not found


<ipython-input-16-c9e483eb357d>:109: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cleaned_data = cleaned_data.applymap(
<ipython-input-16-c9e483eb357d>:109: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cleaned_data = cleaned_data.applymap(
<ipython-input-16-c9e483eb357d>:109: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cleaned_data = cleaned_data.applymap(
<ipython-input-16-c9e483eb357d>:109: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cleaned_data = cleaned_data.applymap(
<ipython-input-16-c9e483eb357d>:109: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cleaned_data = cleaned_data.applymap(
<ipython-input-16-c9e483eb357d>:109: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cleaned_data = cleaned_data.applymap(


Processed and saved: /content/cleaned_reports/22_Sep_2023_Daily_Report_cleaned.csv
Processed and saved: /content/cleaned_reports/25_Sep_2023_Daily_Report_cleaned.csv
Processed and saved: /content/cleaned_reports/3_Nov_2022_Daily_Report._cleaned.csv
Processed and saved: /content/cleaned_reports/12_Oct_2023_Daily_Report_cleaned.csv
Processed and saved: /content/cleaned_reports/9_Aug_2024_Daily_RE_Generation_Report_cleaned.csv
Processed and saved: /content/cleaned_reports/9_Feb_2024_Daily_RE_Report_cleaned.csv


...

Processed and saved: /content/cleaned_reports/19_Nov_2022_Daily_Report._cleaned.csv
Processed and saved: /content/cleaned_reports/01_Sep_2023_Daily_Report_cleaned.csv
Error processing /content/excel_reports/daily26112021.xlsx: Worksheet named 'Generation' not found
Processed and saved: /content/cleaned_reports/12_Aug_2024_Daily_RE_Generation_Report_cleaned.csv
Processed and saved: /content/cleaned_reports/16_Feb_2024_Daily_RE_Report_cleaned.csv
Processed and saved: /content/cleaned_reports/20_Nov_2023_Daily_Report_cleaned.csv


<ipython-input-16-c9e483eb357d>:109: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cleaned_data = cleaned_data.applymap(
<ipython-input-16-c9e483eb357d>:109: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cleaned_data = cleaned_data.applymap(
<ipython-input-16-c9e483eb357d>:109: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cleaned_data = cleaned_data.applymap(


Processed and saved: /content/cleaned_reports/21_May_2024_cleaned.csv
Error processing /content/excel_reports/DailyRE01011970-8.xlsx: Worksheet named 'Generation' not found
Error processing /content/excel_reports/2_Aug.xlsx: Worksheet named 'Generation' not found
Processed and saved: /content/cleaned_reports/6_Nov_2023_Daily_Report_cleaned.csv


<ipython-input-16-c9e483eb357d>:109: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cleaned_data = cleaned_data.applymap(
<ipython-input-16-c9e483eb357d>:109: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cleaned_data = cleaned_data.applymap(
<ipython-input-16-c9e483eb357d>:109: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cleaned_data = cleaned_data.applymap(
<ipython-input-16-c9e483eb357d>:109: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cleaned_data = cleaned_data.applymap(


Processed and saved: /content/cleaned_reports/8_March_2024_cleaned.csv
Processed and saved: /content/cleaned_reports/19_Oct_2023_Daily_Report_cleaned.csv
Processed and saved: /content/cleaned_reports/5_June_2023_Daily_Report_cleaned.csv
Processed and saved: /content/cleaned_reports/31_Dec_2024_Daily_RE_Generation_Report_cleaned.csv
Error processing /content/excel_reports/18_May.xlsx: Worksheet named 'Generation' not found
Error processing /content/excel_reports/12_Nov.xlsx: Worksheet named 'Generation' not found
Data cleaning complete. Cleaned files are saved in the 'cleaned_reports' folder.


In [9]:

# code for merging all resultant files to single file


import os
import pandas as pd

# Define the folder where cleaned files are stored
output_folder = "/content/cleaned_reports"
merged_file_path = "/content/merged_cleaned_reports.csv"

def merge_csv_files(folder_path, output_file):
    # List all cleaned CSV files in the folder
    csv_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith("_cleaned.csv")]
    all_dataframes = []

    print(f"Found {len(csv_files)} cleaned CSV files. Merging them...")

    # Read each CSV file and append it to a list
    for file_path in csv_files:
        try:
            df = pd.read_csv(file_path)
            all_dataframes.append(df)
            print(f"Loaded: {file_path}")
        except Exception as e:
            print(f"Error reading {file_path}: {e}")

    # Concatenate all dataframes
    merged_data = pd.concat(all_dataframes, ignore_index=True)

    # Save the merged dataframe as a single CSV file
    merged_data.to_csv(output_file, index=False)
    print(f"All files merged into: {output_file}")

# Execute the merge
merge_csv_files(output_folder, merged_file_path)



Found 1136 cleaned CSV files. Merging them...
Loaded: /content/cleaned_reports/2_Dec_2022_Daily_Report._cleaned.csv
Loaded: /content/cleaned_reports/10_Nov_2024_Daily_RE_Generation_Report_cleaned.csv
Loaded: /content/cleaned_reports/25_Sep_2023_Daily_Report_cleaned.csv
Loaded: /content/cleaned_reports/DailyRE24112024_cleaned.csv
Loaded: /content/cleaned_reports/16_April_2023_Daily_Report_cleaned.csv
Loaded: /content/cleaned_reports/3_Jan_2023_Daily_Report_cleaned.csv
Loaded: /content/cleaned_reports/2_Jan_2023_Daily_Report_cleaned.csv
Loaded: /content/cleaned_reports/16_Feb_2022_Daily_Report_cleaned.csv
Loaded: /content/cleaned_reports/23_Jan_2025_Daily_RE_Generation_Report_cleaned.csv
Loaded: /content/cleaned_reports/DailyRE26112024_cleaned.csv
Loaded: /content/cleaned_reports/DailyRE15122023_cleaned.csv
Loaded: /content/cleaned_reports/29_Dec_2023_Daily_Report_cleaned.csv
Loaded: /content/cleaned_reports/20_Sep_2024_Daily_RE_Generation_cleaned.csv
Loaded: /content/cleaned_reports/30_

In [15]:
## code to re verify if the resultant csv file is fit for analysis and if needed it can make all the changes what ever required.

import pandas as pd
import numpy as np

# Load the merged CSV file
merged_file_path = "/content/merged_cleaned_reports.csv"
df = pd.read_csv(merged_file_path)

print(f"Initial data shape: {df.shape}")

# 1. Handle missing values
df.replace({"": np.nan, "NA": np.nan, "N/A": np.nan}, inplace=True)
missing_info = df.isna().sum()
print(f"Missing values per column:\n{missing_info}")

# Drop columns with more than 40% missing data (if necessary)
threshold = 0.4 * df.shape[0]
df = df.loc[:, df.isna().sum() < threshold]

# Fill remaining missing numeric values with the mean and categorical with mode
for col in df.columns:
    if df[col].dtype in ['int64', 'float64']:
        df[col].fillna(df[col].mean(), inplace=True)
    else:
        df[col].fillna(df[col].mode()[0], inplace=True)

# 2. Data type conversion
# Convert date columns to datetime
for col in df.columns:
    if "date" in col.lower():
        df[col] = pd.to_datetime(df[col], errors='coerce')

# 3. Column standardization
df.columns = df.columns.str.strip().str.lower().str.replace(r'[^\w\s]', '_')

# 4. Remove outliers (optional)
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_cols:
    upper_limit = df[col].mean() + 3 * df[col].std()
    lower_limit = df[col].mean() - 3 * df[col].std()
    df[col] = np.where(df[col] > upper_limit, upper_limit, df[col])
    df[col] = np.where(df[col] < lower_limit, lower_limit, df[col])

# 5. Handle duplicates
df.drop_duplicates(inplace=True)

# 6. Feature engineering (adding month/year if date is present)
if 'date' in df.columns:
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month

# 7. Save the cleaned and preprocessed data
cleaned_file_path = "/content/final_cleaned_data.csv"
df.to_csv(cleaned_file_path, index=False)
print(f"Cleaned and preprocessed data saved at: {cleaned_file_path}")
print(f"Final data shape: {df.shape}")


Initial data shape: (48552, 9)
Missing values per column:
state_region                  0
wind_energy_mw             1981
solar_energy_mw            1976
other_res_mw               1976
total_mw                   1976
cumulative_wind_mw         1976
cumulative_solar_mw        1976
cumulative_other_res_mw    1976
cumulative_total_mw        1976
dtype: int64


<ipython-input-15-a4f0dcc8ab89>:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)
<ipython-input-15-a4f0dcc8ab89>:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usi

Cleaned and preprocessed data saved at: /content/final_cleaned_data.csv
Final data shape: (30501, 9)
